# s_selection_toy [<img src="https://www.arpm.co/lab/icons/icon_permalink.png" width=30 height=30 style="display: inline;">](https://www.arpm.co/lab/redirect.php?code=s_selection_toy&codeLang=Python)
For details, see [here](https://www.arpm.co/lab/redirect.php?permalink=s_selection_toy).

In [ ]:
import numpy as np
from cvxopt import matrix
from cvxopt import solvers

from arpym.tools.naive_selection import naive_selection
from arpym.tools.forward_selection import forward_selection
from arpym.tools.backward_selection import backward_selection

## [Input parameters](https://www.arpm.co/lab/redirect.php?permalink=s_selection_toy-parameters)

In [ ]:
q2 = np.diag(np.array([1., 1., 1.]))  # quadratic term of the obj. function
c = np.array([0, 0, 0])  # linear term of the obj. function
a_eq = np.atleast_2d([1., 2., 3.])  # equality constraints
b_eq = np.ones((1, 1))

## [Step 1](https://www.arpm.co/lab/redirect.php?permalink=s_selection_toy-implementation-step01): Inner optimization function

In [ ]:
i_1 = np.arange(1, q2.shape[0]+1)  # pool of candidates
n_ = len(i_1)

def optim(s_k):
    
    # define quadratic objective
    q2_ = matrix(2 * q2, tc='d')
    l_ = matrix(c.reshape(-1, 1), tc='d')

    # define constraints
    constraints_f_s_lhs = a_eq
    constraints_f_s_rhs = b_eq
    if s_k.shape[0] <= 2 and s_k.shape[0] > 0:
        idt = np.eye(3)
        not_in_s_k = np.array([j-1 for j in np.squeeze(i_1)
                               if j not in np.squeeze(s_k)])
        constraints_f_s_lhs = np.vstack((constraints_f_s_lhs, idt[not_in_s_k, :]))
        constraints_f_s_rhs = np.vstack((constraints_f_s_rhs,
                                              np.zeros((not_in_s_k.shape[0], 1))))
    constraints_f_s_lhs = matrix(constraints_f_s_lhs, tc='d')
    constraints_f_s_rhs = matrix(constraints_f_s_rhs, tc='d')

    # solve optimization
    solvers.options['show_progress'] = False
    sol = solvers.qp(q2_, l_, A=constraints_f_s_lhs, b=constraints_f_s_rhs)
    x_k_i = np.array(sol['x'])
    f_x_k_i = x_k_i.T @ q2 @ x_k_i + c @ x_k_i

    return x_k_i, f_x_k_i.item()

## [Step 2](https://www.arpm.co/lab/redirect.php?permalink=s_selection_toy-implementation-step02): Best selection of 1, 2 and 3 elements according to the performance g

In [ ]:
i_1 = np.array([[1], [2], [3]])
i_2 = np.array([[1, 2], [1, 3], [2, 3]])
i_3 = np.array([[1, 2, 3]])
i = [i_1, i_2, i_3]

f_star = []
x_star = []
s_star = []
for k in range(n_):
    f = []
    x = []
    for s_k_i in i[k]:
        all = optim(s_k_i)
        x.append(all[0])
        f.append(all[1])
    optim_indice = np.argmin(f)
    s_k_star = i[k][optim_indice]
    all = optim(s_k_star)
    x_star.append(all[0])
    f_star.append(all[1])
    s_star.append(s_k_star)

## [Step 3](https://www.arpm.co/lab/redirect.php?permalink=s_selection_toy-implementation-step03): Optimal number of elements using the naive selection routine

In [ ]:
x_naive, f_x_naive, s_naive = naive_selection(optim, n_)

## [Step 4](https://www.arpm.co/lab/redirect.php?permalink=s_selection_toy-implementation-step04): Optimal number of elements using the forward selection routine

In [ ]:
x_fwd, f_x_fwd, s_fwd = forward_selection(optim, n_)

## [Step 5](https://www.arpm.co/lab/redirect.php?permalink=s_selection_toy-implementation-step05): Optimal number of elements using the backward selection routine

In [ ]:
x_bwd, f_x_bwd, s_bwd = backward_selection(optim, n_)